* To set up, run: mamba activate oggm_env
* Using oggm_env (Python 3.12.2) kernel 

In [2]:
from oggm import cfg
# It will download a small subset of data used for testing and calibration 
# Those data are located in your home directory, in a hidden folder called .oggm

cfg.initialize()
# It will create a configuration file in your home folder, where you can indicate where you want to store further input data. 
# This configuration file is also located in your home directory under the name .oggm_config.

2024-04-04 22:18:17: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-04-04 22:18:17: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-04-04 22:18:17: oggm.cfg: Multiprocessing: using all available processors (N=2)


In [3]:
# To locate this config file: (prints location: '/home/codespace/.oggm_config')
cfg.CONFIG_FILE

'/home/codespace/.oggm_config'

In [4]:
from oggm import cfg, utils
cfg.initialize(logging_level='WARNING')
# A call to cfg.initialize() will read the default parameter file (or any user-provided file) and
# make them available to all other OGGM tools via the cfg.PARAMS dictionary. 

2024-04-04 22:18:22: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-04-04 22:18:22: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-04-04 22:18:22: oggm.cfg: Multiprocessing: using all available processors (N=2)


In [5]:
cfg.PARAMS['melt_f'], cfg.PARAMS['ice_density'], cfg.PARAMS['continue_on_error']

(5.0, 900.0, False)

In [6]:
# You can try with or without multiprocessing: with two glaciers, OGGM could run on two processors
cfg.PARAMS['use_multiprocessing'] = True

2024-04-04 22:22:21: oggm.cfg: Multiprocessing switched ON after user settings.


## Working through a mock workflow

Each OGGM run needs a single folder where to store the results of the computations for all glaciers. This is called a “working directory” and needs to be specified before each run. Here we create a temporary folder for you:

In [7]:
# create temporary folder as working directory 
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-GettingStarted-10m', reset=True)
cfg.PATHS['working_dir']

# creating a permanant folder as working directory: 
# e.g. /home/shruti/OGGM_output

'/tmp/OGGM/OGGM-GettingStarted-10m'

In the OGGM design, this working directory is meant to be persistent, at least as long as you need the data. For example, you can stop your processing workflow after any task, and restart from an existing working directory at a later stage, simply by using the same working directory.

In [9]:
# define the glaciers for the run 
# example glaciers from the tutorial 
rgi_ids = ['RGI60-11.01328', 'RGI60-11.00897'] 
# main glacier on nevados de chillán
glaciar_nevado = ['RGI60-17.13045']

For an operational run on an RGI region, you might want to download the Randolph Glacier Inventory dataset instead, and start a run from it. This case is covered in the working with the RGI tutorial.

## Glacier directories

The OGGM workflow is organized as a list of tasks that have to be applied to a list of glaciers. The vast majority of tasks are called entity tasks: they are standalone operations to be realized on one single glacier entity. These tasks are executed sequentially (one after another): they often need input generated by the previous task(s): for example, the climate calibration needs the glacier flowlines, which can be only computed after the topography data has been processed, and so on.

To handle this situation, OGGM uses a workflow based on data persistence on disk: instead of passing data as python variables from one task to another, each task will read the data from disk and then write the computation results back to the disk, making these new data available for the next task in the queue. These glacier specific data are located in glacier directories.

In [11]:
# One main advantage of this workflow is that OGGM can prepare data and make it available to everyone! Here is an example of an url where such data can be found:
from oggm import workflow
from oggm import DEFAULT_BASE_URL
DEFAULT_BASE_URL

'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.3/elev_bands/W5E5_spinup'

In [12]:
# Let’s use OGGM to download the glacier directories for our two selected glaciers:
gdirs = workflow.init_glacier_directories(
    rgi_ids,  # which glaciers?
    prepro_base_url=DEFAULT_BASE_URL,  # where to fetch the data?
    from_prepro_level=4,  # what kind of data? 
    prepro_border=80  # how big of a map?
)

2024-04-04 22:34:24: oggm.workflow: init_glacier_directories from prepro level 4 on 2 glaciers.
2024-04-04 22:34:24: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 2 glaciers
  0% of 261.6 MiB |                      | Elapsed Time: 0:00:00 ETA:  --:--:--
100% of 261.6 MiB |######################| Elapsed Time: 0:00:09 Time:  0:00:09
  0% of 267.5 MiB |                      | Elapsed Time: 0:00:00 ETA:  --:--:--
100% of 267.5 MiB |######################| Elapsed Time: 0:00:10 Time:  0:00:10


In [26]:
# What if we duplicate the above cell for glaciar nevado? 
nevado_gdir = workflow.init_glacier_directories(
    glaciar_nevado,  # which glaciers?
    prepro_base_url=DEFAULT_BASE_URL,  # where to fetch the data?
    from_prepro_level=4,  # what kind of data? 
    prepro_border=80  # how big of a map?
)

2024-04-04 22:43:54: oggm.workflow: init_glacier_directories from prepro level 4 on 1 glaciers.
2024-04-04 22:43:54: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 1 glaciers


* the keyword from_prepro_level indicates that we will start from pre-processed directories, i.e. data that are already prepared by the OGGM team. In many cases you will want to start from pre-processed directories, in most case from level 3 or 5. For level 3 and above the model has already been calibrated, so you no longer need to do that yourself and can start rigth away with your simulation. Here we start from level 4 and add some data to the processing in order to demonstrate the OGGM workflow.

* the prepro_border keyword indicates the number of grid points which we’d like to add to each side of the glacier for the local map: the larger the glacier will grow, the larger the border parameter should be. The available pre-processed border values are: 10, 80, 160, 240 (depending on the model set-ups there might be more or less options). These are the fixed map sizes we prepared for you - any other map size will require a full processing (see the further DEM sources example for a tutorial).

In [28]:
# The init_glacier_directories task will allways be the very first task to call for all your OGGM experiments. 
# Let’s see what it gives us back:

type(gdirs), type(gdirs[0])

# gdirs is a list of GlacierDirectory objects (one for each glacier). 
# Glacier directories are used by OGGM as “file and attribute manager” for single glaciers. 

# For nevado 
type(nevado_gdir), type(nevado_gdir[0])

(list, oggm.utils._workflow.GlacierDirectory)

In [29]:
# For example, the model now knows where to find the topography data file for this glacier:
gdir = gdirs[0]  # take Unteraar glacier
print('Path to the DEM:', gdir.get_filepath('dem'))

# path to the glaciar nevado dem: 
nevado_gdir = nevado_gdir[0]
print('Path to the DEM:', nevado_gdir.get_filepath('dem'))
# don't get why i needed the [0] since there's only one glacier in the list but ok.. 

Path to the DEM: /tmp/OGGM/OGGM-GettingStarted-10m/per_glacier/RGI60-17/RGI60-17.13/RGI60-17.13045/dem.tif


In [30]:
# And we can also access some attributes of this glacier:
gdir
nevado_gdir

<oggm.GlacierDirectory>
  RGI id: RGI60-17.13045
  Region: 17: Southern Andes
  Subregion: 17-02: C Andes                         
  Glacier type: Ice cap
  Terminus type: Land-terminating
  Status: Glacier or ice cap
  Area: 1.256 km2
  Lon, Lat: (-71.4149, -36.8356)
  Grid (nx, ny): (219, 217)
  Grid (dx, dy): (26.0, -26.0)

In [31]:
gdir.rgi_date  # date at which the outlines are valid
nevado_gdir.rgi_date
# nevado outline only valid for 2000, quite old/out of date 

2000